In [ ]:
from pathlib import Path
from glob import glob
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys

## Visualization for FLAIR:

In [ ]:
p = sorted(list(Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/FLAIR/').glob('*.dcm')), key=lambda path: int(path.stem.rsplit('-')[-1]))

In [ ]:
# credit: https://pydicom.github.io/pydicom/dev/auto_examples/image_processing/reslice.html
# load the DICOM files
def plot_brain_3d(file):
    files = []
    for fname in file:
        files.append(pydicom.dcmread(fname))
    print("file count: {}".format(len(files)))

    # skip files with no SliceLocation (eg scout views)
    slices = []
    skipcount = 0
    for f in files:
        if hasattr(f, 'SliceLocation'):
            slices.append(f)
        else:
            skipcount = skipcount + 1

    print("skipped, no SliceLocation: {}".format(skipcount))

    # ensure they are in the correct order
    slices = sorted(slices, key=lambda s: s.SliceLocation)

    # pixel aspects, assuming all slices are the same
    ps = slices[0].PixelSpacing
    ss = slices[0].SliceThickness
    ax_aspect = ps[1]/ps[0]
    sag_aspect = ps[1]/ss
    cor_aspect = ss/ps[0]

    # create 3D array
    img_shape = list(slices[0].pixel_array.shape)
    img_shape.append(len(slices))
    img3d = np.zeros(img_shape)

    # fill 3D array with the images from the files
    for i, s in enumerate(slices):
        img2d = s.pixel_array
        img3d[:, :, i] = img2d


    # plot 3 orthogonal slices
    plt.figure(figsize=(25,25))
    print("\033[1mBrain CT Scan in three different views (Horizontal, sagittal, coronal)")

    a1 = plt.subplot(1, 3, 1)
    plt.imshow(img3d[:, :, img_shape[2]//2], cmap='hot')
    a1.set_aspect(ax_aspect)

    # plt.figure(figsize=(15,15))
    a2 = plt.subplot(1, 3, 2)
    plt.imshow(img3d[:, img_shape[1]//2, :], cmap='hot')
    a2.set_aspect(sag_aspect)

    # plt.figure(figsize=(15,15))
    a3 = plt.subplot(1, 3, 3)
    plt.imshow(img3d[img_shape[0]//2, :, :].T, cmap='hot')
    a3.set_aspect(cor_aspect)

    # plt.show()

In [ ]:
plot_brain_3d(p)

<p style="color:red">Will be updating this notebook with EDA and more visualization<\p>